# CLIP Guided Stable Diffusion Community Sample
Test edildi. Kisa denemelerde yararli bir fark gorulmedi

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

In [ ]:
models = ['runwayml/stable-diffusion-v1-5']
device = "cuda"

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(models[0], safety_checker=None)
pipe = pipe.to(device, torch.float16)

prompt = "a beautiful pink cat"
negative_prompt = "(worst quality, low quality:2), monochrome, zombie,overexposure, watermark,text,bad anatomy,bad hand,extra hands,extra fingers,too many fingers,fused fingers,bad arm,distorted arm,extra arms,fused arms,extra legs,missing leg,disembodied leg,extra nipples, detached arm, liquid hand,inverted hand,disembodied limb, small breasts, loli, oversized head,extra body,completely nude, extra navel,easynegative,(hair between eyes),sketch, duplicate, ugly, huge eyes, text, logo, worst face, (bad and mutated hands:1.3),  (blurry:2.0), horror, geometry, bad_prompt, (bad hands), (missing fingers), multiple limbs, bad anatomy, (interlocked fingers:1.2), Ugly Fingers, (extra digit and hands and fingers and legs and arms:1.4), ((2girl)), (deformed fingers:1.2), (long fingers:1.2),(bad-artist-anime), bad-artist, bad hand, extra legs ,(ng_deepnegative_v1_75t)"


In [ ]:
generator = torch.Generator(device="cuda").manual_seed(2)
image = pipe(prompt,
             num_inference_steps = 50,
             height = 512,
             width = 512,
             guidance_scale = 6,
             negative_prompt = negative_prompt,
             generator = generator
            ).images[0]

image.save('normal_image.png')

In [ ]:
from transformers import CLIPImageProcessor, CLIPModel
from diffusers import DiffusionPipeline
feature_extractor = CLIPImageProcessor.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K")
clip_model = CLIPModel.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K", torch_dtype=torch.float16)
generator = torch.Generator(device="cuda").manual_seed(2)

guided_pipeline = DiffusionPipeline.from_pretrained(models[0],
                                                    custom_pipeline="clip_guided_stable_diffusion",
                                                    clip_model=clip_model,
                                                    feature_extractor=feature_extractor,    
                                                    torch_dtype=torch.float16)
guided_pipeline.enable_attention_slicing()
guided_pipeline = guided_pipeline.to("cuda")

image = guided_pipeline(prompt,    
                        num_inference_steps=50,
                        guidance_scale=7.5,
                        clip_guidance_scale=6,
                        num_cutouts=4,
                        use_cutouts=False,
                        generator=generator).images[0]
image.save("guided_image.png")